# Random forest classification (single-node, GPU)

<table>
    <tr>
        <td>
            <img src="../_img/rapids.svg" width="300">
        </td>
        <td>
            <img src="../_img/snowflake.png" width="300">
        </td>
    </tr>
</table>

This notebook describes a machine learning training workflow using the famous [NYC Taxi Dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page). That dataset contains information on taxi trips in New York City.

In this exercise, you'll use `cudf` to load a subset of the data from Snowflake and `cuml` to answer this classification question:

> based on characteristics that can be known at the beginning of a trip, will this trip result in a high tip?

## Use RAPIDS libraries

RAPIDS is a collection of libraries which enable you to take advantage of NVIDIA GPUs to accelerate machine learning workflows. This exercise uses the following RAPIDS packages:
    
* [`cudf`](https://github.com/rapidsai/cudf): data frame manipulation, similar to `pandas` and `numpy`
* [`cuml`](https://github.com/rapidsai/cuml): machine learning training and evaluation, similar to `scikit-learn`

These libraries are already available in the `saturncloud/saturn-gpu` images by default. For more information on RAPIDS, see ["Getting Started"](https://rapids.ai/start.html) in the RAPIDS docs.

### Monitor Resource Usage

This tutorial aims to teach you how to take advantage of the GPU for data science workflows. To prove to yourself that RAPIDS is utilizing the GPU, it's important to understand how to monitor that utilization while your code is running. If you already know how to do that, skip to the next section.

<details><summary>(click here to learn how to monitor resource utilization)</summary>

<br>

**Monitoring CPU and Main Memory**
    
CPUs and GPUs are two different types of processors, and the GPU has its own dedicated memory. Many data science libraries that claim to offer GPU acceleration accomplish their tasks with a mix of CPU and GPU use, so it's important to monitor both to see what that code is doing.
    
To monitor CPU utilization and the amount of free main memory (memory available to the CPU), you can use `htop`.
    
Open a new terminal and run `htop`. That will keep an auto-updating dashboard up that shows the CPU utilization and memory usage.
    
**Monitoring GPU and GPU memory**
    
Open a new terminal and run the following command.

```shell
watch -n 5 nvidia-smi
```

<br>
This command will update the output in the terminal every 5 seconds. It shows some information like:

* current CUDA version
* NVIDIA driver version
* internal temperature
* current utilization of GPU memory
* list of processes (if any) currently running on the GPU, and how much GPU memory they're consuming

If you'd prefer a simpler view, you can also consider `gpustat`, which simply tracks temperature, GPU utilization, and memory. This is not available by default in the Saturn GPU images, but you can install it from PyPi.

```shell
pip install gpustat    
```

<br>

And then run it

```shell
gpustat -cp --watch
```

<br>

Whichever option you choose, leave these terminals with the monitoring process running while you work, so you can see how the code below uses the available resources.

</details>

<hr>

## Connect to Snowflake

This examples uses data stored in a Snowflake data warehouse. The code below expects the following environment variables to be set. If you haven't set these, return to the Saturn project page to set them.

* `SNOWFLAKE_ACCOUNT`
* `SNOWFLAKE_USER`
* `SNOWFLAKE_PASSWORD`
* `SNOWFLAKE_WAREHOUSE`
* `TAXI_DATABASE`
* `TAXI_SCHEMA`

For more details on these environment variables, see ["Connecting to Snowflake"](https://docs.snowflake.com/en/user-guide/python-connector-example.html#connecting-to-snowflake) in the `snowflake-connector-python` docs.

The `SNOWFLAKE_*` variables should be set up as Saturn credentials. The `TAXI_*` variables can be set on your Jupyter server or overwritten below based on the Snowflake warehouse and schema you used when loading data into Snowflake. Note that in order to update environment variables your Jupyter server will need to be stopped.

In [ ]:
import os
import pandas as pd
import cudf

import snowflake.connector

SNOWFLAKE_ACCOUNT = os.environ["SNOWFLAKE_ACCOUNT"]
SNOWFLAKE_USER = os.environ["SNOWFLAKE_USER"]
SNOWFLAKE_PASSWORD = os.environ["SNOWFLAKE_PASSWORD"]

SNOWFLAKE_WAREHOUSE = os.environ["SNOWFLAKE_WAREHOUSE"]
TAXI_DATABASE = os.environ["TAXI_DATABASE"]
TAXI_SCHEMA = os.environ["TAXI_SCHEMA"]

conn_info = {
    "account": SNOWFLAKE_ACCOUNT,
    "user": SNOWFLAKE_USER,
    "password": SNOWFLAKE_PASSWORD,
    "warehouse": SNOWFLAKE_WAREHOUSE,
    "database": TAXI_DATABASE,
    "schema": TAXI_SCHEMA,
}
conn = snowflake.connector.connect(**conn_info)

<hr>

## Load data

This example is designed to run quickly with small resources. So let's just load a single month of taxi data for training.

The code below loads the data into a `cudf` data frame. This is similar to a `pandas` dataframe, but it lives in GPU memory and most operations on it are done on the GPU.

This example uses Snowflake to handle the hard work of creating new features, then creates a `cudf` data frame with the result.

In [ ]:
query = """
SELECT
    pickup_taxizone_id,
    dropoff_taxizone_id,
    passenger_count,
    DIV0(tip_amount, fare_amount) > 0.2 AS high_tip,
    DAYOFWEEKISO(pickup_datetime) - 1 AS pickup_weekday,
    WEEKOFYEAR(pickup_datetime) AS pickup_weekofyear,
    HOUR(pickup_datetime) AS pickup_hour,
    (pickup_weekday * 24) + pickup_hour AS pickup_week_hour,
    MINUTE(pickup_datetime) AS pickup_minute
FROM taxi_yellow
WHERE
    DATE_TRUNC('MONTH', pickup_datetime) = %s
"""
taxi = conn.cursor().execute(query, "2019-01-01").fetch_pandas_all()
taxi.columns = [x.lower() for x in taxi.columns]
taxi = cudf.from_pandas(taxi)

The code below computes the size of this dataset in memory.

In [ ]:
print(f"Num rows: {len(taxi)}, Size: {taxi.memory_usage(deep=True).sum() / 1e6} MB")

In [ ]:
numeric_feat = [
    "pickup_weekday",
    "pickup_weekofyear",
    "pickup_hour",
    "pickup_week_hour",
    "pickup_minute",
    "passenger_count",
]
categorical_feat = [
    "pickup_taxizone_id",
    "dropoff_taxizone_id",
]
features = numeric_feat + categorical_feat
y_col = "high_tip"

taxi_train = taxi[features + [y_col]]
taxi_train[features] = taxi_train[features].astype("float32").fillna(-1)
taxi_train[y_col] = taxi_train[y_col].astype("int32").fillna(-1)

You can examine the structure of the data with `cudf` commands:

`.head()` = view the first few rows

In [ ]:
taxi_train.head()

`.dtypes` = list all the columns and the type of data in them

In [ ]:
taxi.dtypes

<hr>

## Train a Model

Now that the data have been prepped, it's time to build a model!

For this task, we'll use the `RandomForestClassifier` from `cuml`. If you've never used a random forest or need a refresher, consult ["Forests of randomized trees"](https://scikit-learn.org/stable/modules/ensemble.html#forest) in the `sciki-learn` documentation.

The code below initializes a random forest classifier with the following parameter values.

* `n_estimators=100` = create a 100-tree forest
* `max_depth=10` = stop growing a tree once it contains a leaf node that is 10 levels below the root
* `n_streams=4` = create 4 decision trees at a time
    - setting this to a value higher than 1 can reduce training time, but setting it too high can increase training time
    - increasing this parameter's value increases the memory requirements for training

All other parameters use the defaults from `RandomForestClassifier`.

<details><summary>(click here to learn why data scientists do this)</summary>

**Setting max_depth**

Tree-based models split the training data into smaller and smaller groups, to try to group together records with similar values of the target. A tree can be thought of as a collection of rules like `pickup_hour greater than 11` and `pickup_minute less than 31.0`. As you add more rules, those groups (called "leaf nodes") get smaller. In an extreme example, a model could create a tree with enough rules to place each record in the training data into its own group. That would probably take a lot of rules, and would be referred to as a "deep" tree.

Deep trees are problematic because their descriptions of the world are too specific to be useful on new data. Imagine training a classification model to predict whether or not visitors to a theme park will ride a particular rollercoaster. You could measure the time down to the millisecond that every guest's ticket is scanned at the entrance, and a model might learn a rule like *"if the guest has been to the park before and if the guest is older than 40 and younger than 41, and if the guest is staying at Hotel A and if the guest enters the park after 1:00:17.456 and if the guest enters the park earlier than 1:00:17.995, they will ride the rollercoaster"*. This is very very unlikely to ever match any future visitors, and if it does it's unlikely that this prediction will be very good unless you have some reason to believe that a visitor arriving at 1:00:18 instead of 1:00:17 really changes the probability that they'll ride that rollercoaster.

To prevent this situation (called "overfitting"), most tree-based machine learning algorithms accept parameters that control how deep the trees can get. `max_depth` is common, and says "don't create a rule more complex than this". In the example above, that rule has a depth of 7.

1. visiting the park
2. has been to the park before?
3. older than 40?
4. younger than 41?
5. staying at Hotel A?
6. entered the park after 1:00:17.456?
7. entered the park before 1:00:17.995?

Setting `max_depth = 5` would have prevented those weirdly-specific timing rules from ever being generated.

Choosing good values for this parameter is part art, part science, and is outside the scope of this tutorial.

</details>

In [ ]:
from cuml.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=100, max_depth=10, n_streams=4)

With the classifier created, fit it to some data! The code below uses `%%time` to print out a timing, so you can see how long it takes to train. This can be used to compare `cuml` to methods explored in other notebooks, or to test how changing some parameters to `RandomForestClassifier` changes the runtime for training.

In [ ]:
%%time
_ = rfc.fit(taxi_train[features], taxi_train[y_col])

<hr>

## Save model

Once you've trained a model, save it in a file to use later for scoring or for comparison with other models.

There are several ways to do this, but `cloudpickle` is likely to give you the best experience. It handles some common drawbacks of the built-in `pickle` library.

`cloudpickle` can be used to write a Python object to bytes, and to create a Python object from that binary representation.

In [ ]:
import cloudpickle
import os

MODEL_PATH = "models"
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)

with open(f"{MODEL_PATH}/random_forest_rapids.pkl", "wb") as f:
    cloudpickle.dump(rfc, f)

<hr>

## Calculate metrics on test set

Machine learning training tries to create a model which can produce useful results on new data that it didn't see during training. To test how well we've accomplished that in this example, read in another month of taxi data from Snowflake.

In [ ]:
taxi = conn.cursor().execute(query, "2019-02-01")
columns = [x[0] for x in taxi.description]
# using fetchall() because rapids requires a different pyarrow version than snowflake-connector-python
taxi = pd.DataFrame(taxi.fetchall(), columns=columns)
taxi.columns = taxi.columns.str.lower()
taxi = cudf.from_pandas(taxi)
taxi_test = taxi

In [ ]:
taxi_test[features] = taxi_test[features].astype("float32").fillna(-1)
taxi_test[y_col] = taxi_test[y_col].astype("int32").fillna(-1)

`cuml` comes with many functions for calculating metrics that describe how well a model's predictions match the actual values. For a complete list, see [the cuml API docs](https://docs.rapids.ai/api/cuml/stable/api.html#metrics-regression-classification-and-distance) or run the code below.

In [ ]:
import cuml.metrics

[m for m in dir(cuml.metrics) if not m.startswith("_")]

This tutorial uses the `roc_auc_score` to evaluate the model. This metric measures the area under the [receiver operating characteristic](https://en.wikipedia.org/wiki/Receiver_operating_characteristic) curve. Values closer to 1.0 are desirable.

In [ ]:
from cuml.metrics import roc_auc_score

preds = rfc.predict_proba(taxi_test[features])[1]
roc_auc_score(taxi_test[y_col], preds)

<hr>

## Next Steps

In this tutorial, you learned how to train a model for a binary classification task, using `cuml`, based on data in Snowflake. Training took around 5 seconds for a dataset that was 0.31 GB in memory, a huge improvement on the almost 9 minutes it took to [train the same model using `scikit-learn`](./rf-scikit.ipynb)!

If you wanted to train a much larger model (think `max_depth=16, num_iterations=10000`) or use a much larger dataset or both, it might not be possible on a single machine. Try [this dask-cudf notebook](./rf-rapids-dask.ipynb) to learn how to use Dask to take advantage of multiple-machine, multi-GPU training.

<hr>